In [76]:
import librosa
import matplotlib.pyplot as plt

import librosa.display
import IPython.display as ipd
import numpy as np
import os
sampling_rate = 44100
import spotipy 
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
from savify import Savify
from savify.types import Type, Format, Quality
import logging
from savify.utils import PathHolder
from savify.logger import Logger
import math
import json

In [77]:
CLIENT_ID = "11f43fbbd82448ed9233f4c2d13fe3ea"
CLIENT_SECRET = "e64ac46ed23c41058d9f0a3aa3d7dc80"

auth_manager = SpotifyClientCredentials(client_id = CLIENT_ID, client_secret = CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)
logger = Logger(log_location='path/for/logs', log_level=None) # Silent output




In [78]:
#2500 per each playlist



# playlists:




# 2444
energetic = [["spotify", "37i9dQZF1DX8hY56Fq3fM0?si=7d3230c0243045a5"], 
             ["spotify", "37i9dQZF1DWZVAVMhIe3pV?si=7ed8729e9cf942a3"], 
             ["spotify", "37i9dQZF1DX8hY56Fq3fM0?si=7d3230c0243045a5"], 
             ["spotify", "37i9dQZF1DWZixSclZdoFE?si=89b3a266838b4f22"], 
             ["spotify", "37i9dQZF1DX8hY56Fq3fM0?si=7d3230c0243045a5"], 
             ["spotify", "37i9dQZF1DX0vHZ8elq0UK?si=c0662441d38740ab"], 
             ["spotify", "37i9dQZF1DWUyPpT6WoO7A?si=6d6bb57955974cd3"], 
             ["spotify", "37i9dQZF1DWTUHzPOW6Jl7?si=833fdc89a87d4601"], 
            
             
           ]


# 2506 songs
calm = [["spotify", "37i9dQZF1DX4WYpdgoIcn6?si=1b1c5104d7d44e04"], 
        ["spotify", "37i9dQZF1DX2yvmlOdMYzV?si=d5dc4d4ff4114aa7"], 
        ["spotify", "37i9dQZF1DX6VdMW310YC7?si=71adcb47e32a48d5"],
        ["spotify", "37i9dQZF1DX889U0CL85jj?si=e885fed69e864b5b"], 
        ["spotify", "37i9dQZF1DWWzVPEmatsUB?si=d95653605d3a40d5"], 
        ["spotify", "37i9dQZF1DX1s9knjP51Oa?si=acf58bbff2b9473c"], 
        ["spotify", "37i9dQZF1DXcr2UzLGERUU?si=90443963c5d9480c"]]
 
    
#2455

sad = [
       ["spotify", "37i9dQZF1DXaS0Bb5SeHIi?si=b2ff0b6ea0674e4f"], ["spotify", "37i9dQZF1DWZFicI79sEj9?si=eb24fc501ae84a67"], ["spotify", "37i9dQZF1DXa39zZwdBPSN?si=bc2cd2095e2142d8"], ["spotify", "37i9dQZF1DX8Vz2ROLXhTT?si=34782dfb8d5949e1"], ["spotify", "37i9dQZF1DWSqBruwoIXkA?si=4940b5bb53994515"], ["spotify", "37i9dQZF1DXbm0dp7JzNeL?si=21d2b55a6d424611"]
      
      
      ]




# 2480

happy = [["spotify", "37i9dQZF1DWSf2RDTDayIx?si=d31895dc098144cf"],["spotify", "37i9dQZF1DX7KNKjOK0o75?si=691a472985db4384"],["spotify", "37i9dQZF1DWYBO1MoTDhZI?si=839e1a9252e64e89"], ["spotify", "37i9dQZF1DXdPec7aLTmlC?si=5cc8d996c3644dba"], ["spotify", "37i9dQZF1DX0E9XMGembJo?si=7e9ee9f543464052"], ["spotify", "37i9dQZF1DX3rxVfibe1L0?si=2727adfa469e4fa9"],
        ["spotify", "37i9dQZF1DWSkMjlBZAZ07?si=0615f0e1f02648c8"]

      
     
         
         
         
        
        
       ]

mood_songs = [energetic, calm, sad, happy]





class_length = []



In [4]:
def extract_songs(playlists, mood):
    track_urls = []
    path = 'data/'+ mood
    s = Savify(download_format=Format.WAV, api_credentials=("CLIENT_ID","CLIENT_SECRET"), path_holder=PathHolder(downloads_path=path), logger=logger)
    for playlist in playlists:
        playlist = sp.user_playlist_tracks(playlist[0], playlist[1])
        for item in playlist['tracks']['items']:
        
            track = item['track']
            url = "https://open.spotify.com/track/" + track['id']
            if url not in class_length:
                track_urls.append(url)
                s.download(url)

#extract_songs(energetic, "energetic")
#extract_songs(sad, "sad")
#extract_songs(calm, "calm")
#extract_songs(happy, "happy")
    

In [82]:
# MFCCs

# n_ftt = the number of audio samples per window 

DATASET_PATH = "data"
JSON_PATH = "data.json"
SAMPLE_RATE = 22050
TRACK_DURATION = 30
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION

def pre_process(file, duration):
    signal, sr = librosa.load(file, sr=SAMPLE_RATE)
    trimmed, index = librosa.effects.trim(signal) # trim away silent sections
    final_signal = trimmed[:(sampling_rate * duration)] # trim signal to duration in seconds
    return final_signal, sr



def get_mfccs(dataset_path,json_path, n_mfcc=13, n_ftt = 2048, hop_length=512, num_segments=5):
    # dictionary to store data
    
    data = {
        
        "mapping": [],
        "mfcc": [],
        "labels": []
    }
    
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    expected_vector_length = math.ceil(num_samples_per_segment/hop_length)
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        
        if dirpath is not dataset_path:
            dirpath_components = dirpath.split("/")
            semantic_label = dirpath_components[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing {}".format(semantic_label))
            
            for f in filenames:
                
                file_path = os.path.join(dirpath, f)
                signal, sr = pre_process(file_path, TRACK_DURATION)
                
                for s in range(num_segments):
                    start_sample = num_samples_per_segment * s
                    finish_sample = start_sample + num_samples_per_segment
                    
                    mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample], 
                                               sr=sr, 
                                               n_mfcc=n_mfcc, 
                                               hop_length = hop_length
                                               )
                    mfcc = mfcc.T
                    
                    if len(mfcc) == expected_vector_length:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1)
                        print("{}, segment:{}".format(file_path, s))

                        

    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)

#get_mfccs(DATASET_PATH, JSON_PATH, num_segments=10)

In [119]:
def get_average_zcr(data_set_path, frame_length=1024,hop_length=512, num_segments=5):
    average_zcrs = []
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    
    data = {
        
        "mapping": [],
        "zcr": [],
        "labels": []
    }
   
    i = 0
    for filename in [f for f in os.listdir(data_set_path) if not f.startswith('.')]:
        
        data["mapping"].append(filename)
        for track in os.listdir(data_set_path + "/" + filename)[:200]:
            
            path = data_set_path + "/" + filename + "/" + track
            signal, sr = pre_process(path, TRACK_DURATION)
            for s in range(num_segments):
                start_sample = num_samples_per_segment * s
                finish_sample = start_sample + num_samples_per_segment
                sample_zcr = librosa.feature.zero_crossing_rate(signal[start_sample:finish_sample], frame_length=1024, hop_length=512).tolist()
                data["zcr"].extend(sample_zcr)
                data["labels"].append(i)
                print("Sample: " + str(s) + str(path))
                
        i += 1
    
    with open("zcr_data.json", "w") as fp:
        json.dump(data, fp, indent=4)
    
get_average_zcr("data")


Sample: 0data/happy/Absofacto - Someone Else□s Dream.wav
Sample: 1data/happy/Absofacto - Someone Else□s Dream.wav
Sample: 2data/happy/Absofacto - Someone Else□s Dream.wav
Sample: 3data/happy/Absofacto - Someone Else□s Dream.wav
Sample: 4data/happy/Absofacto - Someone Else□s Dream.wav
Sample: 0data/happy/Thomas Headon - Bored.wav
Sample: 1data/happy/Thomas Headon - Bored.wav
Sample: 2data/happy/Thomas Headon - Bored.wav
Sample: 3data/happy/Thomas Headon - Bored.wav
Sample: 4data/happy/Thomas Headon - Bored.wav
Sample: 0data/happy/Clubhouse - Weekend.wav
Sample: 1data/happy/Clubhouse - Weekend.wav
Sample: 2data/happy/Clubhouse - Weekend.wav
Sample: 3data/happy/Clubhouse - Weekend.wav
Sample: 4data/happy/Clubhouse - Weekend.wav
Sample: 0data/happy/John Summit - Better Than This.wav
Sample: 1data/happy/John Summit - Better Than This.wav
Sample: 2data/happy/John Summit - Better Than This.wav
Sample: 3data/happy/John Summit - Better Than This.wav
Sample: 4data/happy/John Summit - Better Than

Sample: 0data/happy/Jason Derulo - Take You Dancing.wav
Sample: 1data/happy/Jason Derulo - Take You Dancing.wav
Sample: 2data/happy/Jason Derulo - Take You Dancing.wav
Sample: 3data/happy/Jason Derulo - Take You Dancing.wav
Sample: 4data/happy/Jason Derulo - Take You Dancing.wav
Sample: 0data/happy/Stevie Wonder - Sir Duke.wav
Sample: 1data/happy/Stevie Wonder - Sir Duke.wav
Sample: 2data/happy/Stevie Wonder - Sir Duke.wav
Sample: 3data/happy/Stevie Wonder - Sir Duke.wav
Sample: 4data/happy/Stevie Wonder - Sir Duke.wav
Sample: 0data/happy/Becky Hill - Remember.wav
Sample: 1data/happy/Becky Hill - Remember.wav
Sample: 2data/happy/Becky Hill - Remember.wav
Sample: 3data/happy/Becky Hill - Remember.wav
Sample: 4data/happy/Becky Hill - Remember.wav
Sample: 0data/happy/BENEE - Glitter.wav
Sample: 1data/happy/BENEE - Glitter.wav
Sample: 2data/happy/BENEE - Glitter.wav
Sample: 3data/happy/BENEE - Glitter.wav
Sample: 4data/happy/BENEE - Glitter.wav
Sample: 0data/happy/ODIE - Miss Summer.wav
Sa

Sample: 0data/happy/Kygo - I'll Wait.wav
Sample: 1data/happy/Kygo - I'll Wait.wav
Sample: 2data/happy/Kygo - I'll Wait.wav
Sample: 3data/happy/Kygo - I'll Wait.wav
Sample: 4data/happy/Kygo - I'll Wait.wav
Sample: 0data/happy/Elton John - Tiny Dancer.wav
Sample: 1data/happy/Elton John - Tiny Dancer.wav
Sample: 2data/happy/Elton John - Tiny Dancer.wav
Sample: 3data/happy/Elton John - Tiny Dancer.wav
Sample: 4data/happy/Elton John - Tiny Dancer.wav
Sample: 0data/happy/Nicky Romero - Nights With You.wav
Sample: 1data/happy/Nicky Romero - Nights With You.wav
Sample: 2data/happy/Nicky Romero - Nights With You.wav
Sample: 3data/happy/Nicky Romero - Nights With You.wav
Sample: 4data/happy/Nicky Romero - Nights With You.wav
Sample: 0data/happy/Ruel - Painkiller.wav
Sample: 1data/happy/Ruel - Painkiller.wav
Sample: 2data/happy/Ruel - Painkiller.wav
Sample: 3data/happy/Ruel - Painkiller.wav
Sample: 4data/happy/Ruel - Painkiller.wav
Sample: 0data/happy/JP Cooper - Little Bit Of Love - Indian Summe

Sample: 0data/happy/Toploader - Dancing in the Moonlight.wav
Sample: 1data/happy/Toploader - Dancing in the Moonlight.wav
Sample: 2data/happy/Toploader - Dancing in the Moonlight.wav
Sample: 3data/happy/Toploader - Dancing in the Moonlight.wav
Sample: 4data/happy/Toploader - Dancing in the Moonlight.wav
Sample: 0data/happy/Grady - SO GOOD.wav
Sample: 1data/happy/Grady - SO GOOD.wav
Sample: 2data/happy/Grady - SO GOOD.wav
Sample: 3data/happy/Grady - SO GOOD.wav
Sample: 4data/happy/Grady - SO GOOD.wav
Sample: 0data/happy/Clément Leroux - Baby (Tell Me).wav
Sample: 1data/happy/Clément Leroux - Baby (Tell Me).wav
Sample: 2data/happy/Clément Leroux - Baby (Tell Me).wav
Sample: 3data/happy/Clément Leroux - Baby (Tell Me).wav
Sample: 4data/happy/Clément Leroux - Baby (Tell Me).wav
Sample: 0data/happy/Kota the Friend - Long Beach.wav
Sample: 1data/happy/Kota the Friend - Long Beach.wav
Sample: 2data/happy/Kota the Friend - Long Beach.wav
Sample: 3data/happy/Kota the Friend - Long Beach.wav
Sam

Sample: 0data/happy/Arden Jones - Parallel Parking.wav
Sample: 1data/happy/Arden Jones - Parallel Parking.wav
Sample: 2data/happy/Arden Jones - Parallel Parking.wav
Sample: 3data/happy/Arden Jones - Parallel Parking.wav
Sample: 4data/happy/Arden Jones - Parallel Parking.wav
Sample: 0data/happy/Tungevaag - Kingdoms.wav
Sample: 1data/happy/Tungevaag - Kingdoms.wav
Sample: 2data/happy/Tungevaag - Kingdoms.wav
Sample: 3data/happy/Tungevaag - Kingdoms.wav
Sample: 4data/happy/Tungevaag - Kingdoms.wav
Sample: 0data/happy/Keith Urban - One Too Many.wav
Sample: 1data/happy/Keith Urban - One Too Many.wav
Sample: 2data/happy/Keith Urban - One Too Many.wav
Sample: 3data/happy/Keith Urban - One Too Many.wav
Sample: 4data/happy/Keith Urban - One Too Many.wav
Sample: 0data/happy/Sam Feldt - Everything About You (feat. your friend polly).wav
Sample: 1data/happy/Sam Feldt - Everything About You (feat. your friend polly).wav
Sample: 2data/happy/Sam Feldt - Everything About You (feat. your friend polly).

...

Sample: 0data/calm/Justin Bieber - Holy (feat. Chance The Rapper).wav
Sample: 1data/calm/Justin Bieber - Holy (feat. Chance The Rapper).wav
Sample: 2data/calm/Justin Bieber - Holy (feat. Chance The Rapper).wav
Sample: 3data/calm/Justin Bieber - Holy (feat. Chance The Rapper).wav
Sample: 4data/calm/Justin Bieber - Holy (feat. Chance The Rapper).wav
Sample: 0data/calm/Lauv - Feelings.wav
Sample: 1data/calm/Lauv - Feelings.wav
Sample: 2data/calm/Lauv - Feelings.wav
Sample: 3data/calm/Lauv - Feelings.wav
Sample: 4data/calm/Lauv - Feelings.wav
Sample: 0data/calm/Leaving Laurel - Winter In The Woods.wav
Sample: 1data/calm/Leaving Laurel - Winter In The Woods.wav
Sample: 2data/calm/Leaving Laurel - Winter In The Woods.wav
Sample: 3data/calm/Leaving Laurel - Winter In The Woods.wav
Sample: 4data/calm/Leaving Laurel - Winter In The Woods.wav
Sample: 0data/calm/Mac Ayres - Where U Goin' Tonight□.wav
Sample: 1data/calm/Mac Ayres - Where U Goin' Tonight□.wav
Sample: 2data/calm/Mac Ayres - Where U 

Sample: 0data/calm/ASTU - Motion.wav
Sample: 1data/calm/ASTU - Motion.wav
Sample: 2data/calm/ASTU - Motion.wav
Sample: 3data/calm/ASTU - Motion.wav
Sample: 4data/calm/ASTU - Motion.wav
Sample: 0data/calm/Adele - Easy On Me.wav
Sample: 1data/calm/Adele - Easy On Me.wav
Sample: 2data/calm/Adele - Easy On Me.wav
Sample: 3data/calm/Adele - Easy On Me.wav
Sample: 4data/calm/Adele - Easy On Me.wav
Sample: 0data/calm/Alexander 23 - Nothing's the Same.wav
Sample: 1data/calm/Alexander 23 - Nothing's the Same.wav
Sample: 2data/calm/Alexander 23 - Nothing's the Same.wav
Sample: 3data/calm/Alexander 23 - Nothing's the Same.wav
Sample: 4data/calm/Alexander 23 - Nothing's the Same.wav
Sample: 0data/calm/Son Little - Mad About You.wav
Sample: 1data/calm/Son Little - Mad About You.wav
Sample: 2data/calm/Son Little - Mad About You.wav
Sample: 3data/calm/Son Little - Mad About You.wav
Sample: 4data/calm/Son Little - Mad About You.wav
Sample: 0data/calm/Pink Sweat$ - 17.wav
Sample: 1data/calm/Pink Sweat$

Sample: 0data/calm/Zen Boba - Talk to Yuh.wav
Sample: 1data/calm/Zen Boba - Talk to Yuh.wav
Sample: 2data/calm/Zen Boba - Talk to Yuh.wav
Sample: 3data/calm/Zen Boba - Talk to Yuh.wav
Sample: 4data/calm/Zen Boba - Talk to Yuh.wav
Sample: 0data/calm/Victoria Monét - Coastin□.wav
Sample: 1data/calm/Victoria Monét - Coastin□.wav
Sample: 2data/calm/Victoria Monét - Coastin□.wav
Sample: 3data/calm/Victoria Monét - Coastin□.wav
Sample: 4data/calm/Victoria Monét - Coastin□.wav
Sample: 0data/calm/Lukas Graham - Happy For You.wav
Sample: 1data/calm/Lukas Graham - Happy For You.wav
Sample: 2data/calm/Lukas Graham - Happy For You.wav
Sample: 3data/calm/Lukas Graham - Happy For You.wav
Sample: 4data/calm/Lukas Graham - Happy For You.wav
Sample: 0data/calm/JP Saxe - Line By Line (feat. Maren Morris).wav
Sample: 1data/calm/JP Saxe - Line By Line (feat. Maren Morris).wav
Sample: 2data/calm/JP Saxe - Line By Line (feat. Maren Morris).wav
Sample: 3data/calm/JP Saxe - Line By Line (feat. Maren Morris).w

Sample: 0data/calm/rum.gold - Waiting For (feat. Jamila Woods).wav
Sample: 1data/calm/rum.gold - Waiting For (feat. Jamila Woods).wav
Sample: 2data/calm/rum.gold - Waiting For (feat. Jamila Woods).wav
Sample: 3data/calm/rum.gold - Waiting For (feat. Jamila Woods).wav
Sample: 4data/calm/rum.gold - Waiting For (feat. Jamila Woods).wav
Sample: 0data/calm/Joesef - Loverboy.wav
Sample: 1data/calm/Joesef - Loverboy.wav
Sample: 2data/calm/Joesef - Loverboy.wav
Sample: 3data/calm/Joesef - Loverboy.wav
Sample: 4data/calm/Joesef - Loverboy.wav
Sample: 0data/calm/Harry Styles - Falling.wav
Sample: 1data/calm/Harry Styles - Falling.wav
Sample: 2data/calm/Harry Styles - Falling.wav
Sample: 3data/calm/Harry Styles - Falling.wav
Sample: 4data/calm/Harry Styles - Falling.wav
Sample: 0data/calm/Taylor Swift - exile (feat. Bon Iver).wav
Sample: 1data/calm/Taylor Swift - exile (feat. Bon Iver).wav
Sample: 2data/calm/Taylor Swift - exile (feat. Bon Iver).wav
Sample: 3data/calm/Taylor Swift - exile (feat. 

Sample: 0data/calm/Clinton Kane - i don't want to watch the world end with someone else.wav
Sample: 1data/calm/Clinton Kane - i don't want to watch the world end with someone else.wav
Sample: 2data/calm/Clinton Kane - i don't want to watch the world end with someone else.wav
Sample: 3data/calm/Clinton Kane - i don't want to watch the world end with someone else.wav
Sample: 4data/calm/Clinton Kane - i don't want to watch the world end with someone else.wav
Sample: 0data/calm/PIE - Apathy.wav
Sample: 1data/calm/PIE - Apathy.wav
Sample: 2data/calm/PIE - Apathy.wav
Sample: 3data/calm/PIE - Apathy.wav
Sample: 4data/calm/PIE - Apathy.wav
Sample: 0data/calm/Gabrielle Aplin - Losing Me.wav
Sample: 1data/calm/Gabrielle Aplin - Losing Me.wav
Sample: 2data/calm/Gabrielle Aplin - Losing Me.wav
Sample: 3data/calm/Gabrielle Aplin - Losing Me.wav
Sample: 4data/calm/Gabrielle Aplin - Losing Me.wav
Sample: 0data/calm/Charlotte Lawrence - Slow Motion.wav
Sample: 1data/calm/Charlotte Lawrence - Slow Moti

In [94]:
print(type(librosa.feature.zero_crossing_rate(signal, frame_length=1024, hop_length=512)[0]))




<class 'numpy.ndarray'>


In [126]:
def amplitude_envelope(signal, frame_size, hop_length):
    amplitude_env = []
    for i in range(0, len(signal), hop_length):
        current_frame_amp_env = max(signal[i:i + frame_size])
        amplitude_env.append(current_frame_amp_env)
    return np.array(amplitude_env)

signal, sr = pre_process("data/happy/FLETCHER - Cherry (feat. Hayley Kiyoko).wav", 30)
signal_ae = amplitude_envelope(signal[0:int(SAMPLES_PER_TRACK / 5)],1024, 512)

In [128]:
print(len(signal_ae))

259


In [131]:
def get_rms(data_set_path, frame_length=1024,hop_length=512, num_segments=5):
    average_zcrs = []
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    
    data = {
        
        "mapping": [],
        "rms": [],
        "labels": []
    }
   
    i = 0
    for filename in [f for f in os.listdir(data_set_path) if not f.startswith('.')]:
        
        data["mapping"].append(filename)
        for track in os.listdir(data_set_path + "/" + filename)[:200]:
            
            path = data_set_path + "/" + filename + "/" + track
            signal, sr = pre_process(path, TRACK_DURATION)
            for s in range(num_segments):
                start_sample = num_samples_per_segment * s
                finish_sample = start_sample + num_samples_per_segment
                sample_rms = librosa.feature.rms(signal[start_sample:finish_sample], frame_length=1024, hop_length=512).tolist()
                data["rms"].extend(sample_rms)
                data["labels"].append(i)
                print("Sample: " + str(s) + str(path))
                
        i += 1
    
    with open("rms_data.json", "w") as fp:
        json.dump(data, fp, indent=4)
    
get_rms("data")

Sample: 0data/happy/Absofacto - Someone Else□s Dream.wav
Sample: 1data/happy/Absofacto - Someone Else□s Dream.wav
Sample: 2data/happy/Absofacto - Someone Else□s Dream.wav
Sample: 3data/happy/Absofacto - Someone Else□s Dream.wav
Sample: 4data/happy/Absofacto - Someone Else□s Dream.wav
Sample: 0data/happy/Thomas Headon - Bored.wav
Sample: 1data/happy/Thomas Headon - Bored.wav
Sample: 2data/happy/Thomas Headon - Bored.wav
Sample: 3data/happy/Thomas Headon - Bored.wav
Sample: 4data/happy/Thomas Headon - Bored.wav
Sample: 0data/happy/Clubhouse - Weekend.wav
Sample: 1data/happy/Clubhouse - Weekend.wav
Sample: 2data/happy/Clubhouse - Weekend.wav
Sample: 3data/happy/Clubhouse - Weekend.wav
Sample: 4data/happy/Clubhouse - Weekend.wav
Sample: 0data/happy/John Summit - Better Than This.wav
Sample: 1data/happy/John Summit - Better Than This.wav
Sample: 2data/happy/John Summit - Better Than This.wav
Sample: 3data/happy/John Summit - Better Than This.wav
Sample: 4data/happy/John Summit - Better Than

Sample: 0data/happy/Jason Derulo - Take You Dancing.wav
Sample: 1data/happy/Jason Derulo - Take You Dancing.wav
Sample: 2data/happy/Jason Derulo - Take You Dancing.wav
Sample: 3data/happy/Jason Derulo - Take You Dancing.wav
Sample: 4data/happy/Jason Derulo - Take You Dancing.wav
Sample: 0data/happy/Stevie Wonder - Sir Duke.wav
Sample: 1data/happy/Stevie Wonder - Sir Duke.wav
Sample: 2data/happy/Stevie Wonder - Sir Duke.wav
Sample: 3data/happy/Stevie Wonder - Sir Duke.wav
Sample: 4data/happy/Stevie Wonder - Sir Duke.wav
Sample: 0data/happy/Becky Hill - Remember.wav
Sample: 1data/happy/Becky Hill - Remember.wav
Sample: 2data/happy/Becky Hill - Remember.wav
Sample: 3data/happy/Becky Hill - Remember.wav
Sample: 4data/happy/Becky Hill - Remember.wav
Sample: 0data/happy/BENEE - Glitter.wav
Sample: 1data/happy/BENEE - Glitter.wav
Sample: 2data/happy/BENEE - Glitter.wav
Sample: 3data/happy/BENEE - Glitter.wav
Sample: 4data/happy/BENEE - Glitter.wav
Sample: 0data/happy/ODIE - Miss Summer.wav
Sa

Sample: 0data/happy/Kygo - I'll Wait.wav
Sample: 1data/happy/Kygo - I'll Wait.wav
Sample: 2data/happy/Kygo - I'll Wait.wav
Sample: 3data/happy/Kygo - I'll Wait.wav
Sample: 4data/happy/Kygo - I'll Wait.wav
Sample: 0data/happy/Elton John - Tiny Dancer.wav
Sample: 1data/happy/Elton John - Tiny Dancer.wav
Sample: 2data/happy/Elton John - Tiny Dancer.wav
Sample: 3data/happy/Elton John - Tiny Dancer.wav
Sample: 4data/happy/Elton John - Tiny Dancer.wav
Sample: 0data/happy/Nicky Romero - Nights With You.wav
Sample: 1data/happy/Nicky Romero - Nights With You.wav
Sample: 2data/happy/Nicky Romero - Nights With You.wav
Sample: 3data/happy/Nicky Romero - Nights With You.wav
Sample: 4data/happy/Nicky Romero - Nights With You.wav
Sample: 0data/happy/Ruel - Painkiller.wav
Sample: 1data/happy/Ruel - Painkiller.wav
Sample: 2data/happy/Ruel - Painkiller.wav
Sample: 3data/happy/Ruel - Painkiller.wav
Sample: 4data/happy/Ruel - Painkiller.wav
Sample: 0data/happy/JP Cooper - Little Bit Of Love - Indian Summe

Sample: 0data/happy/Toploader - Dancing in the Moonlight.wav
Sample: 1data/happy/Toploader - Dancing in the Moonlight.wav
Sample: 2data/happy/Toploader - Dancing in the Moonlight.wav
Sample: 3data/happy/Toploader - Dancing in the Moonlight.wav
Sample: 4data/happy/Toploader - Dancing in the Moonlight.wav
Sample: 0data/happy/Grady - SO GOOD.wav
Sample: 1data/happy/Grady - SO GOOD.wav
Sample: 2data/happy/Grady - SO GOOD.wav
Sample: 3data/happy/Grady - SO GOOD.wav
Sample: 4data/happy/Grady - SO GOOD.wav
Sample: 0data/happy/Clément Leroux - Baby (Tell Me).wav
Sample: 1data/happy/Clément Leroux - Baby (Tell Me).wav
Sample: 2data/happy/Clément Leroux - Baby (Tell Me).wav
Sample: 3data/happy/Clément Leroux - Baby (Tell Me).wav
Sample: 4data/happy/Clément Leroux - Baby (Tell Me).wav
Sample: 0data/happy/Kota the Friend - Long Beach.wav
Sample: 1data/happy/Kota the Friend - Long Beach.wav
Sample: 2data/happy/Kota the Friend - Long Beach.wav
Sample: 3data/happy/Kota the Friend - Long Beach.wav
Sam

Sample: 0data/happy/Arden Jones - Parallel Parking.wav
Sample: 1data/happy/Arden Jones - Parallel Parking.wav
Sample: 2data/happy/Arden Jones - Parallel Parking.wav
Sample: 3data/happy/Arden Jones - Parallel Parking.wav
Sample: 4data/happy/Arden Jones - Parallel Parking.wav
Sample: 0data/happy/Tungevaag - Kingdoms.wav
Sample: 1data/happy/Tungevaag - Kingdoms.wav
Sample: 2data/happy/Tungevaag - Kingdoms.wav
Sample: 3data/happy/Tungevaag - Kingdoms.wav
Sample: 4data/happy/Tungevaag - Kingdoms.wav
Sample: 0data/happy/Keith Urban - One Too Many.wav
Sample: 1data/happy/Keith Urban - One Too Many.wav
Sample: 2data/happy/Keith Urban - One Too Many.wav
Sample: 3data/happy/Keith Urban - One Too Many.wav
Sample: 4data/happy/Keith Urban - One Too Many.wav
Sample: 0data/happy/Sam Feldt - Everything About You (feat. your friend polly).wav
Sample: 1data/happy/Sam Feldt - Everything About You (feat. your friend polly).wav
Sample: 2data/happy/Sam Feldt - Everything About You (feat. your friend polly).

...

Sample: 0data/calm/Justin Bieber - Holy (feat. Chance The Rapper).wav
Sample: 1data/calm/Justin Bieber - Holy (feat. Chance The Rapper).wav
Sample: 2data/calm/Justin Bieber - Holy (feat. Chance The Rapper).wav
Sample: 3data/calm/Justin Bieber - Holy (feat. Chance The Rapper).wav
Sample: 4data/calm/Justin Bieber - Holy (feat. Chance The Rapper).wav
Sample: 0data/calm/Lauv - Feelings.wav
Sample: 1data/calm/Lauv - Feelings.wav
Sample: 2data/calm/Lauv - Feelings.wav
Sample: 3data/calm/Lauv - Feelings.wav
Sample: 4data/calm/Lauv - Feelings.wav
Sample: 0data/calm/Leaving Laurel - Winter In The Woods.wav
Sample: 1data/calm/Leaving Laurel - Winter In The Woods.wav
Sample: 2data/calm/Leaving Laurel - Winter In The Woods.wav
Sample: 3data/calm/Leaving Laurel - Winter In The Woods.wav
Sample: 4data/calm/Leaving Laurel - Winter In The Woods.wav
Sample: 0data/calm/Mac Ayres - Where U Goin' Tonight□.wav
Sample: 1data/calm/Mac Ayres - Where U Goin' Tonight□.wav
Sample: 2data/calm/Mac Ayres - Where U 

Sample: 0data/calm/ASTU - Motion.wav
Sample: 1data/calm/ASTU - Motion.wav
Sample: 2data/calm/ASTU - Motion.wav
Sample: 3data/calm/ASTU - Motion.wav
Sample: 4data/calm/ASTU - Motion.wav
Sample: 0data/calm/Adele - Easy On Me.wav
Sample: 1data/calm/Adele - Easy On Me.wav
Sample: 2data/calm/Adele - Easy On Me.wav
Sample: 3data/calm/Adele - Easy On Me.wav
Sample: 4data/calm/Adele - Easy On Me.wav
Sample: 0data/calm/Alexander 23 - Nothing's the Same.wav
Sample: 1data/calm/Alexander 23 - Nothing's the Same.wav
Sample: 2data/calm/Alexander 23 - Nothing's the Same.wav
Sample: 3data/calm/Alexander 23 - Nothing's the Same.wav
Sample: 4data/calm/Alexander 23 - Nothing's the Same.wav
Sample: 0data/calm/Son Little - Mad About You.wav
Sample: 1data/calm/Son Little - Mad About You.wav
Sample: 2data/calm/Son Little - Mad About You.wav
Sample: 3data/calm/Son Little - Mad About You.wav
Sample: 4data/calm/Son Little - Mad About You.wav
Sample: 0data/calm/Pink Sweat$ - 17.wav
Sample: 1data/calm/Pink Sweat$

Sample: 0data/calm/Zen Boba - Talk to Yuh.wav
Sample: 1data/calm/Zen Boba - Talk to Yuh.wav
Sample: 2data/calm/Zen Boba - Talk to Yuh.wav
Sample: 3data/calm/Zen Boba - Talk to Yuh.wav
Sample: 4data/calm/Zen Boba - Talk to Yuh.wav
Sample: 0data/calm/Victoria Monét - Coastin□.wav
Sample: 1data/calm/Victoria Monét - Coastin□.wav
Sample: 2data/calm/Victoria Monét - Coastin□.wav
Sample: 3data/calm/Victoria Monét - Coastin□.wav
Sample: 4data/calm/Victoria Monét - Coastin□.wav
Sample: 0data/calm/Lukas Graham - Happy For You.wav
Sample: 1data/calm/Lukas Graham - Happy For You.wav
Sample: 2data/calm/Lukas Graham - Happy For You.wav
Sample: 3data/calm/Lukas Graham - Happy For You.wav
Sample: 4data/calm/Lukas Graham - Happy For You.wav
Sample: 0data/calm/JP Saxe - Line By Line (feat. Maren Morris).wav
Sample: 1data/calm/JP Saxe - Line By Line (feat. Maren Morris).wav
Sample: 2data/calm/JP Saxe - Line By Line (feat. Maren Morris).wav
Sample: 3data/calm/JP Saxe - Line By Line (feat. Maren Morris).w

Sample: 0data/calm/rum.gold - Waiting For (feat. Jamila Woods).wav
Sample: 1data/calm/rum.gold - Waiting For (feat. Jamila Woods).wav
Sample: 2data/calm/rum.gold - Waiting For (feat. Jamila Woods).wav
Sample: 3data/calm/rum.gold - Waiting For (feat. Jamila Woods).wav
Sample: 4data/calm/rum.gold - Waiting For (feat. Jamila Woods).wav
Sample: 0data/calm/Joesef - Loverboy.wav
Sample: 1data/calm/Joesef - Loverboy.wav
Sample: 2data/calm/Joesef - Loverboy.wav
Sample: 3data/calm/Joesef - Loverboy.wav
Sample: 4data/calm/Joesef - Loverboy.wav
Sample: 0data/calm/Harry Styles - Falling.wav
Sample: 1data/calm/Harry Styles - Falling.wav
Sample: 2data/calm/Harry Styles - Falling.wav
Sample: 3data/calm/Harry Styles - Falling.wav
Sample: 4data/calm/Harry Styles - Falling.wav
Sample: 0data/calm/Taylor Swift - exile (feat. Bon Iver).wav
Sample: 1data/calm/Taylor Swift - exile (feat. Bon Iver).wav
Sample: 2data/calm/Taylor Swift - exile (feat. Bon Iver).wav
Sample: 3data/calm/Taylor Swift - exile (feat. 

Sample: 0data/calm/Clinton Kane - i don't want to watch the world end with someone else.wav
Sample: 1data/calm/Clinton Kane - i don't want to watch the world end with someone else.wav
Sample: 2data/calm/Clinton Kane - i don't want to watch the world end with someone else.wav
Sample: 3data/calm/Clinton Kane - i don't want to watch the world end with someone else.wav
Sample: 4data/calm/Clinton Kane - i don't want to watch the world end with someone else.wav
Sample: 0data/calm/PIE - Apathy.wav
Sample: 1data/calm/PIE - Apathy.wav
Sample: 2data/calm/PIE - Apathy.wav
Sample: 3data/calm/PIE - Apathy.wav
Sample: 4data/calm/PIE - Apathy.wav
Sample: 0data/calm/Gabrielle Aplin - Losing Me.wav
Sample: 1data/calm/Gabrielle Aplin - Losing Me.wav
Sample: 2data/calm/Gabrielle Aplin - Losing Me.wav
Sample: 3data/calm/Gabrielle Aplin - Losing Me.wav
Sample: 4data/calm/Gabrielle Aplin - Losing Me.wav
Sample: 0data/calm/Charlotte Lawrence - Slow Motion.wav
Sample: 1data/calm/Charlotte Lawrence - Slow Moti

In [143]:
def get_tempo(data_set_path, frame_length=1024,hop_length=512, num_segments=5):
    average_zcrs = []
    num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    
    data = {
        
        "mapping": [],
        "tempo": [],
        "labels": []
    }
   
    i = 0
    for filename in [f for f in os.listdir(data_set_path) if not f.startswith('.')]:
        
        data["mapping"].append(filename)
        for track in os.listdir(data_set_path + "/" + filename)[:200]:
            
            path = data_set_path + "/" + filename + "/" + track
            signal, sr = pre_process(path, TRACK_DURATION)
            for s in range(num_segments):
                start_sample = num_samples_per_segment * s
                finish_sample = start_sample + num_samples_per_segment
                onset_env = librosa.onset.onset_strength(signal[start_sample:finish_sample], sr=sr)
                d_tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr,aggregate=None)
                data["tempo"].extend(d_tempo)
                data["labels"].append(i)
                print("Sample: " + str(s) + str(path))
                
        i += 1
    
    with open("tempo_data.json", "w") as fp:
        json.dump(data, fp, indent=4)
    
get_tempo("data")





Sample: 0data/happy/Absofacto - Someone Else□s Dream.wav
Sample: 1data/happy/Absofacto - Someone Else□s Dream.wav
Sample: 2data/happy/Absofacto - Someone Else□s Dream.wav
Sample: 3data/happy/Absofacto - Someone Else□s Dream.wav
Sample: 4data/happy/Absofacto - Someone Else□s Dream.wav
Sample: 0data/happy/Thomas Headon - Bored.wav
Sample: 1data/happy/Thomas Headon - Bored.wav
Sample: 2data/happy/Thomas Headon - Bored.wav
Sample: 3data/happy/Thomas Headon - Bored.wav
Sample: 4data/happy/Thomas Headon - Bored.wav
Sample: 0data/happy/Clubhouse - Weekend.wav
Sample: 1data/happy/Clubhouse - Weekend.wav
Sample: 2data/happy/Clubhouse - Weekend.wav
Sample: 3data/happy/Clubhouse - Weekend.wav
Sample: 4data/happy/Clubhouse - Weekend.wav
Sample: 0data/happy/John Summit - Better Than This.wav
Sample: 1data/happy/John Summit - Better Than This.wav
Sample: 2data/happy/John Summit - Better Than This.wav
Sample: 3data/happy/John Summit - Better Than This.wav
Sample: 4data/happy/John Summit - Better Than

Sample: 0data/happy/Jason Derulo - Take You Dancing.wav
Sample: 1data/happy/Jason Derulo - Take You Dancing.wav
Sample: 2data/happy/Jason Derulo - Take You Dancing.wav
Sample: 3data/happy/Jason Derulo - Take You Dancing.wav
Sample: 4data/happy/Jason Derulo - Take You Dancing.wav
Sample: 0data/happy/Stevie Wonder - Sir Duke.wav
Sample: 1data/happy/Stevie Wonder - Sir Duke.wav
Sample: 2data/happy/Stevie Wonder - Sir Duke.wav
Sample: 3data/happy/Stevie Wonder - Sir Duke.wav
Sample: 4data/happy/Stevie Wonder - Sir Duke.wav
Sample: 0data/happy/Becky Hill - Remember.wav
Sample: 1data/happy/Becky Hill - Remember.wav
Sample: 2data/happy/Becky Hill - Remember.wav
Sample: 3data/happy/Becky Hill - Remember.wav
Sample: 4data/happy/Becky Hill - Remember.wav
Sample: 0data/happy/BENEE - Glitter.wav
Sample: 1data/happy/BENEE - Glitter.wav
Sample: 2data/happy/BENEE - Glitter.wav
Sample: 3data/happy/BENEE - Glitter.wav
Sample: 4data/happy/BENEE - Glitter.wav
Sample: 0data/happy/ODIE - Miss Summer.wav
Sa

Sample: 0data/happy/Kygo - I'll Wait.wav
Sample: 1data/happy/Kygo - I'll Wait.wav
Sample: 2data/happy/Kygo - I'll Wait.wav
Sample: 3data/happy/Kygo - I'll Wait.wav
Sample: 4data/happy/Kygo - I'll Wait.wav
Sample: 0data/happy/Elton John - Tiny Dancer.wav
Sample: 1data/happy/Elton John - Tiny Dancer.wav
Sample: 2data/happy/Elton John - Tiny Dancer.wav
Sample: 3data/happy/Elton John - Tiny Dancer.wav
Sample: 4data/happy/Elton John - Tiny Dancer.wav
Sample: 0data/happy/Nicky Romero - Nights With You.wav
Sample: 1data/happy/Nicky Romero - Nights With You.wav
Sample: 2data/happy/Nicky Romero - Nights With You.wav
Sample: 3data/happy/Nicky Romero - Nights With You.wav
Sample: 4data/happy/Nicky Romero - Nights With You.wav
Sample: 0data/happy/Ruel - Painkiller.wav
Sample: 1data/happy/Ruel - Painkiller.wav
Sample: 2data/happy/Ruel - Painkiller.wav
Sample: 3data/happy/Ruel - Painkiller.wav
Sample: 4data/happy/Ruel - Painkiller.wav
Sample: 0data/happy/JP Cooper - Little Bit Of Love - Indian Summe

Sample: 0data/happy/Toploader - Dancing in the Moonlight.wav
Sample: 1data/happy/Toploader - Dancing in the Moonlight.wav
Sample: 2data/happy/Toploader - Dancing in the Moonlight.wav
Sample: 3data/happy/Toploader - Dancing in the Moonlight.wav
Sample: 4data/happy/Toploader - Dancing in the Moonlight.wav
Sample: 0data/happy/Grady - SO GOOD.wav
Sample: 1data/happy/Grady - SO GOOD.wav
Sample: 2data/happy/Grady - SO GOOD.wav
Sample: 3data/happy/Grady - SO GOOD.wav
Sample: 4data/happy/Grady - SO GOOD.wav
Sample: 0data/happy/Clément Leroux - Baby (Tell Me).wav
Sample: 1data/happy/Clément Leroux - Baby (Tell Me).wav
Sample: 2data/happy/Clément Leroux - Baby (Tell Me).wav
Sample: 3data/happy/Clément Leroux - Baby (Tell Me).wav
Sample: 4data/happy/Clément Leroux - Baby (Tell Me).wav
Sample: 0data/happy/Kota the Friend - Long Beach.wav
Sample: 1data/happy/Kota the Friend - Long Beach.wav
Sample: 2data/happy/Kota the Friend - Long Beach.wav
Sample: 3data/happy/Kota the Friend - Long Beach.wav
Sam

Sample: 0data/happy/Arden Jones - Parallel Parking.wav
Sample: 1data/happy/Arden Jones - Parallel Parking.wav
Sample: 2data/happy/Arden Jones - Parallel Parking.wav
Sample: 3data/happy/Arden Jones - Parallel Parking.wav
Sample: 4data/happy/Arden Jones - Parallel Parking.wav
Sample: 0data/happy/Tungevaag - Kingdoms.wav
Sample: 1data/happy/Tungevaag - Kingdoms.wav
Sample: 2data/happy/Tungevaag - Kingdoms.wav
Sample: 3data/happy/Tungevaag - Kingdoms.wav
Sample: 4data/happy/Tungevaag - Kingdoms.wav
Sample: 0data/happy/Keith Urban - One Too Many.wav
Sample: 1data/happy/Keith Urban - One Too Many.wav
Sample: 2data/happy/Keith Urban - One Too Many.wav
Sample: 3data/happy/Keith Urban - One Too Many.wav
Sample: 4data/happy/Keith Urban - One Too Many.wav
Sample: 0data/happy/Sam Feldt - Everything About You (feat. your friend polly).wav
Sample: 1data/happy/Sam Feldt - Everything About You (feat. your friend polly).wav
Sample: 2data/happy/Sam Feldt - Everything About You (feat. your friend polly).

...

Sample: 0data/calm/Justin Bieber - Holy (feat. Chance The Rapper).wav
Sample: 1data/calm/Justin Bieber - Holy (feat. Chance The Rapper).wav
Sample: 2data/calm/Justin Bieber - Holy (feat. Chance The Rapper).wav
Sample: 3data/calm/Justin Bieber - Holy (feat. Chance The Rapper).wav
Sample: 4data/calm/Justin Bieber - Holy (feat. Chance The Rapper).wav
Sample: 0data/calm/Lauv - Feelings.wav
Sample: 1data/calm/Lauv - Feelings.wav
Sample: 2data/calm/Lauv - Feelings.wav
Sample: 3data/calm/Lauv - Feelings.wav
Sample: 4data/calm/Lauv - Feelings.wav
Sample: 0data/calm/Leaving Laurel - Winter In The Woods.wav
Sample: 1data/calm/Leaving Laurel - Winter In The Woods.wav
Sample: 2data/calm/Leaving Laurel - Winter In The Woods.wav
Sample: 3data/calm/Leaving Laurel - Winter In The Woods.wav
Sample: 4data/calm/Leaving Laurel - Winter In The Woods.wav
Sample: 0data/calm/Mac Ayres - Where U Goin' Tonight□.wav
Sample: 1data/calm/Mac Ayres - Where U Goin' Tonight□.wav
Sample: 2data/calm/Mac Ayres - Where U 

Sample: 0data/calm/ASTU - Motion.wav
Sample: 1data/calm/ASTU - Motion.wav
Sample: 2data/calm/ASTU - Motion.wav
Sample: 3data/calm/ASTU - Motion.wav
Sample: 4data/calm/ASTU - Motion.wav
Sample: 0data/calm/Adele - Easy On Me.wav
Sample: 1data/calm/Adele - Easy On Me.wav
Sample: 2data/calm/Adele - Easy On Me.wav
Sample: 3data/calm/Adele - Easy On Me.wav
Sample: 4data/calm/Adele - Easy On Me.wav
Sample: 0data/calm/Alexander 23 - Nothing's the Same.wav
Sample: 1data/calm/Alexander 23 - Nothing's the Same.wav
Sample: 2data/calm/Alexander 23 - Nothing's the Same.wav
Sample: 3data/calm/Alexander 23 - Nothing's the Same.wav
Sample: 4data/calm/Alexander 23 - Nothing's the Same.wav
Sample: 0data/calm/Son Little - Mad About You.wav
Sample: 1data/calm/Son Little - Mad About You.wav
Sample: 2data/calm/Son Little - Mad About You.wav
Sample: 3data/calm/Son Little - Mad About You.wav
Sample: 4data/calm/Son Little - Mad About You.wav
Sample: 0data/calm/Pink Sweat$ - 17.wav
Sample: 1data/calm/Pink Sweat$

Sample: 0data/calm/Zen Boba - Talk to Yuh.wav
Sample: 1data/calm/Zen Boba - Talk to Yuh.wav
Sample: 2data/calm/Zen Boba - Talk to Yuh.wav
Sample: 3data/calm/Zen Boba - Talk to Yuh.wav
Sample: 4data/calm/Zen Boba - Talk to Yuh.wav
Sample: 0data/calm/Victoria Monét - Coastin□.wav
Sample: 1data/calm/Victoria Monét - Coastin□.wav
Sample: 2data/calm/Victoria Monét - Coastin□.wav
Sample: 3data/calm/Victoria Monét - Coastin□.wav
Sample: 4data/calm/Victoria Monét - Coastin□.wav
Sample: 0data/calm/Lukas Graham - Happy For You.wav
Sample: 1data/calm/Lukas Graham - Happy For You.wav
Sample: 2data/calm/Lukas Graham - Happy For You.wav
Sample: 3data/calm/Lukas Graham - Happy For You.wav
Sample: 4data/calm/Lukas Graham - Happy For You.wav
Sample: 0data/calm/JP Saxe - Line By Line (feat. Maren Morris).wav
Sample: 1data/calm/JP Saxe - Line By Line (feat. Maren Morris).wav
Sample: 2data/calm/JP Saxe - Line By Line (feat. Maren Morris).wav
Sample: 3data/calm/JP Saxe - Line By Line (feat. Maren Morris).w

Sample: 0data/calm/rum.gold - Waiting For (feat. Jamila Woods).wav
Sample: 1data/calm/rum.gold - Waiting For (feat. Jamila Woods).wav
Sample: 2data/calm/rum.gold - Waiting For (feat. Jamila Woods).wav
Sample: 3data/calm/rum.gold - Waiting For (feat. Jamila Woods).wav
Sample: 4data/calm/rum.gold - Waiting For (feat. Jamila Woods).wav
Sample: 0data/calm/Joesef - Loverboy.wav
Sample: 1data/calm/Joesef - Loverboy.wav
Sample: 2data/calm/Joesef - Loverboy.wav
Sample: 3data/calm/Joesef - Loverboy.wav
Sample: 4data/calm/Joesef - Loverboy.wav
Sample: 0data/calm/Harry Styles - Falling.wav
Sample: 1data/calm/Harry Styles - Falling.wav
Sample: 2data/calm/Harry Styles - Falling.wav
Sample: 3data/calm/Harry Styles - Falling.wav
Sample: 4data/calm/Harry Styles - Falling.wav
Sample: 0data/calm/Taylor Swift - exile (feat. Bon Iver).wav
Sample: 1data/calm/Taylor Swift - exile (feat. Bon Iver).wav
Sample: 2data/calm/Taylor Swift - exile (feat. Bon Iver).wav
Sample: 3data/calm/Taylor Swift - exile (feat. 

Sample: 0data/calm/Clinton Kane - i don't want to watch the world end with someone else.wav
Sample: 1data/calm/Clinton Kane - i don't want to watch the world end with someone else.wav
Sample: 2data/calm/Clinton Kane - i don't want to watch the world end with someone else.wav
Sample: 3data/calm/Clinton Kane - i don't want to watch the world end with someone else.wav
Sample: 4data/calm/Clinton Kane - i don't want to watch the world end with someone else.wav
Sample: 0data/calm/PIE - Apathy.wav
Sample: 1data/calm/PIE - Apathy.wav
Sample: 2data/calm/PIE - Apathy.wav
Sample: 3data/calm/PIE - Apathy.wav
Sample: 4data/calm/PIE - Apathy.wav
Sample: 0data/calm/Gabrielle Aplin - Losing Me.wav
Sample: 1data/calm/Gabrielle Aplin - Losing Me.wav
Sample: 2data/calm/Gabrielle Aplin - Losing Me.wav
Sample: 3data/calm/Gabrielle Aplin - Losing Me.wav
Sample: 4data/calm/Gabrielle Aplin - Losing Me.wav
Sample: 0data/calm/Charlotte Lawrence - Slow Motion.wav
Sample: 1data/calm/Charlotte Lawrence - Slow Moti

In [141]:
print(len(dtempo))

2584
